<a href="https://colab.research.google.com/github/ZPavlo/GazeLandmarksHourGlass/blob/main/eye_landmarks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

from google.colab import drive
drive.mount('/content/gdrive')
ROOT_DATA = 'gdrive/MyDrive/unityeye/'

Mounted at /content/gdrive


In [2]:
import os, sys

!git clone https://github.com/ZPavlo/GazeLandmarksHourGlass 
sys.path.append("GazeLandmarksHourGlass")

Cloning into 'GazeLandmarksHourGlass'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 81 (delta 34), reused 63 (delta 22), pack-reused 0
Unpacking objects: 100% (81/81), done.


In [3]:
!pip install progress

  Created wheel for progress: filename=progress-1.5-cp36-none-any.whl size=8074 sha256=cc2812883fdf9afa1c355cad950d9f419a3f0d24f131915a290fec9bdb06ec56
  Stored in directory: /root/.cache/pip/wheels/6c/c8/80/32a294e3041f006c661838c05a411c7b7ffc60ff939d14e116
Successfully built progress


In [4]:
import torch 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
from model.hourglass import HourglassNet

NUM_STACKS = 3
NUM_BLOCKS = 4
NUM_CLASSES = 17
print("==> creating model: stacks={}, blocks={}".format(NUM_STACKS, NUM_BLOCKS))
model = HourglassNet(num_stacks=NUM_STACKS, num_blocks=NUM_BLOCKS, num_classes=NUM_CLASSES)
model = model.to(device)

==> creating model: stacks=3, blocks=4


In [6]:
from losses import JointsMSELoss
from utils.optimizer import get_optimizer

LR = 0.0001

criterion = JointsMSELoss().to(device)
optimizer = get_optimizer("adam", model, lr=LR)

In [7]:
from utils.data_preprocessing import EyeLandmarksDataset, TrainDataset, TestDataset
from torch.utils.data import DataLoader

unity_eye = EyeLandmarksDataset(ROOT_DATA, load_full=True)
train_dataset = TrainDataset(unity_eye, data_config={
    "max_shift": (5, 7),
    "delta_scale": 0.4,
    "max_rotation_angle": 0.5,
    "image_size": (128, 96),
    "line_count": 2,
    "down_up_scale": 0.4,
    "sigma_head_map": 35.0,
})

test_dataset = TestDataset(unity_eye, {
    "line_count": 0,
    "image_size": (128, 96),
})

100%|██████████| 10/10 [02:15<00:00, 13.59s/it]


***********************************
Max shift: (5, 7)
Delta scale: 0.4
Max rotation angle: 0.5
Image size: (128, 96)
Line count: 2
Down up scale: 0.4
Sigma head map: 35.0
***********************************
***********************************
Max shift: (3, 4)
Delta scale: 0.0
Max rotation angle: 0.3
Image size: (128, 96)
Line count: 0
Down up scale: 0.0
Sigma head map: 1.0
***********************************


In [8]:
from train import train, validate
from utils.tools import adjust_learning_rate, save_checkpoint
lr = LR

lrs = []
train_losses = []
valid_losses = []
train_acc_list = []
valid_acc_list = []
start_epoch = 0
best_acc = 0.0

In [9]:
BATCH_SIZE = 16
NUM_WORKERS = 1

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

In [12]:
import numpy as np

EPOCH_SIZE = 5
SNAPSHOT = 1
CHECKPOINT_PATH = r"/content/gdrive/MyDrive/exp1"

difficult = np.linspace(0.15, 0.95, EPOCH_SIZE)

SCHEDULE = [3, 6, 9]
GAMMA = 0.8

for epoch in range(start_epoch, EPOCH_SIZE):
    lr = adjust_learning_rate(optimizer, epoch, lr, SCHEDULE, GAMMA)
    print('\nEpoch: %d | LR: %.8f' % (epoch + 1, lr))

    train_dataset.set_difficult(difficult[epoch])
    test_dataset.set_difficult(difficult[epoch])
    # train for one epoch
    train_loss, train_acc, best_acc = train(train_dataloader, model, criterion, optimizer, print_step=10,
                                            save_step=900, checkpoint_path=CHECKPOINT_PATH, best_acc=best_acc)

    # evaluate on validation set
    valid_loss, valid_acc, best_acc = validate(test_dataloader, model, criterion, print_step=10, best_acc=best_acc)

    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_acc': max(valid_acc, best_acc),
        'optimizer': optimizer.state_dict(),
    }, valid_acc > best_acc, checkpoint=CHECKPOINT_PATH, snapshot=SNAPSHOT)



Epoch: 1 | LR: 0.00010000
(10/2657) Batch: 1.265s | Total: 0:00:12 | ETA: 0:56:10 | Loss: 3.10e-01 | Acc:  0.652,  0.967
(20/2657) Batch: 1.310s | Total: 0:00:25 | ETA: 0:56:17 | Loss: 2.89e-01 | Acc:  0.643,  0.952
(30/2657) Batch: 1.251s | Total: 0:00:38 | ETA: 0:55:02 | Loss: 2.69e-01 | Acc:  0.618,  0.956
(40/2657) Batch: 1.236s | Total: 0:00:50 | ETA: 0:54:01 | Loss: 2.52e-01 | Acc:  0.713,  0.996
(50/2657) Batch: 1.209s | Total: 0:01:02 | ETA: 0:53:45 | Loss: 2.36e-01 | Acc:  0.660,  0.949
(60/2657) Batch: 1.155s | Total: 0:01:14 | ETA: 0:51:23 | Loss: 2.22e-01 | Acc:  0.691,  0.971
(70/2657) Batch: 1.340s | Total: 0:01:26 | ETA: 0:51:52 | Loss: 2.09e-01 | Acc:  0.698,  0.967
(80/2657) Batch: 1.170s | Total: 0:01:39 | ETA: 0:54:20 | Loss: 1.98e-01 | Acc:  0.646,  0.982
(90/2657) Batch: 1.190s | Total: 0:01:51 | ETA: 0:52:25 | Loss: 1.88e-01 | Acc:  0.594,  0.952
(100/2657) Batch: 1.233s | Total: 0:02:03 | ETA: 0:52:42 | Loss: 1.78e-01 | Acc:  0.611,  0.949
(110/2657) Batch: 1.31

FileNotFoundError: ignored

In [ ]:
print(train_losses)
print(valid_losses)
print(train_acc_list)
print(valid_acc_list)